In [7]:
# coding=utf-8
from __future__ import print_function, absolute_import 
from gm.api import *

In [8]:
import numpy as np
import pandas as pd

In [9]:
set_token('9c0950e38c59552734328ad13ad93b6cc44ee271')
# 基础交易数据

In [10]:
# context.symbol = 'SHSE.510300'
start_date = '2018-02-28'
end_date =  '2020-02-28'

In [11]:
trade_data = history('SHSE.510300', frequency='1d', start_time=start_date, end_time=end_date, fill_missing='last',
                         df=True)
# 去除 'symbol', 'eob', 'frequency','position' 列
trade_data.drop(['symbol', 'eob', 'frequency','position'], axis=1, inplace=True)
# 将'bob'去时区化后作为索引
trade_data.set_index('bob', inplace=True)
# 将data的索引设置为tz-naive
trade_data.index = trade_data.index.tz_localize(None)


In [12]:

trade_data

In [13]:
data = trade_data
data.info()

# 添加输出/监督变量：未来T日的平均收益率

In [14]:
def add_return_column(data, T=3):
    """
    向DataFrame添加未来T日的平均日收益率。

    参数:
    - data: 包含交易数据的DataFrame。
    - T: 未来的天数，默认为3。

    返回值:
    - 该函数没有返回值，但会修改传入的DataFrame，为每个交易日添加一个新列，表示未来T日的收益率。
    """
    # (1+avg)^T = (1+y) = close(T)/close 
    # 取对数，T ln(1+avg) = ln(1+y)
    # 对于较小量，ln(1+x) ≈ x
    # 所以 avg = y/T =(close(T)/close-1)/T
    data['avg_daily_return_'+str(T)] = (data['close'].shift(-T)/data['close']-1)/T

In [15]:
test = data
for T in np.arange(1,31,1):
    add_return_column(test,T)
test.info()

In [16]:
test.describe()

In [28]:
import matplotlib.pyplot as plt
# 计算所有 avg_daily_return_T 的均值
column_means = data.filter(regex='avg_daily_return_').mean()

# 绘制扇形图
plt.figure(figsize=(10, 8))
plt.pie(column_means, labels=column_means.index, autopct='%1.1f%%', startangle=90)
plt.title('Average Daily Returns T Mean Distribution')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

In [18]:
T = 3
add_return_column(data,T)
data.info()

# 添加技术指标

In [10]:
def add_ma_columns(data, ma_periods):
    """
    向DataFrame添加移动平均列。

    参数:
    - data: 包含交易数据的DataFrame。
    - ma_periods: 一个包含要计算的移动平均周期的列表，每个元素是一个整数，表示天数。

    返回值:
    - 该函数没有返回值，但会修改传入的DataFrame，为每个指定的周期添加一个新列。
    """
    for period in ma_periods:
        ma_column_name = f'MA_{period}'  # 根据周期生成列名称，如'MA_5'表示5天移动平均
        data[ma_column_name] = data['close'].rolling(window=period).mean()

def add_ema_columns(data, ema_periods):
    """
    向DataFrame添加指数移动平均列。

    参数:
    - data: 包含交易数据的DataFrame。
    - ema_periods: 一个包含要计算的指数移动平均周期的列表，每个元素是一个整数，表示天数。

    返回值:
    - 该函数没有返回值，但会修改传入的DataFrame，为每个指定的周期添加一个新列。
    """
    for period in ema_periods:
        ema_column_name = f'EMA_{period}'  # 根据周期生成列名称，如'EMA_5'表示5天指数移动平均
        data[ema_column_name] = data['close'].ewm(span=period).mean()

def add_rsi_factor(data, period=14):
    """
    向DataFrame添加相对强弱指数（RSI）因子。

    参数:
    - data: 包含交易数据的DataFrame。
    - period: 计算RSI的周期，默认为14天。
    """
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))

def add_atr_factor(data, period=14):
    """
    向DataFrame添加平均真实范围（ATR）因子。

    参数:
    - data: 包含交易数据的DataFrame。
    - period: 计算ATR的周期，默认为14天。
    """
    high_low = data['high'] - data['low']
    high_close = (data['high'] - data['close'].shift()).abs()
    low_close = (data['low'] - data['close'].shift()).abs()
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = ranges.max(axis=1)
    data['ATR'] = true_range.rolling(window=period).mean()

def add_bollinger_band_width_factor(data, period=20, num_std=2):
    """
    向DataFrame添加布林带宽度因子。

    参数:
    - data: 包含交易数据的DataFrame。
    - period: 计算布林带的周期，默认为20天。
    - num_std: 布林带的标准差倍数，默认为2。
    """
    ma = data['close'].rolling(window=period).mean()
    std = data['close'].rolling(window=period).std()
    upper_band = ma + (std * num_std)
    lower_band = ma - (std * num_std)
    data['Bollinger_Width'] = upper_band - lower_band

def add_vwap_factor(data):
    """
    向DataFrame添加成交量加权平均价格（VWAP）因子。

    参数:
    - data: 包含交易数据的DataFrame。
    """
    data['VWAP'] = (data['volume'] * data['close']).cumsum() / data['volume'].cumsum()


In [11]:
add_ma_columns(data, [5, 10, 20, 60, 120])
add_ema_columns(data, [5, 10, 20, 60, 120])
add_rsi_factor(data)
add_atr_factor(data)
add_bollinger_band_width_factor(data)
add_vwap_factor(data)
data.info()

# 添加EMD分解 (库安装有问题，废弃)

In [1]:
from PyEMD import EMD


In [4]:
def add_emd_columns(data, baseColumn):
    """
    向DataFrame添加EMD分解列。

    参数:
    - data: 包含交易数据的DataFrame。
    - baseColumn: 一个字符串，表示要分解的基础列。

    返回值:
    - 该函数没有返回值，但会修改传入的DataFrame，为每个IMF和剩余部分添加一个新列。
    """
    
    emd = EMD()
    imfs = emd(data[baseColumn].values)
    for i, imf in enumerate(imfs):
        data[f'IMF_{i+1}'] = imf
    data['Residual'] = data[baseColumn] - sum(imfs)
        
add_emd_columns(data, 'close')

# 宏观数据指标

In [179]:
macro_data = pd.read_excel('../data/macro.xlsx')

In [180]:
#date作为索引
macro_data.set_index('date', inplace=True)

In [181]:
# 查看date是否为tz-aware
try:
    macro_data.index.tz_localize('UTC')
    print("The 'date' index is not tz-aware.")
except TypeError:
    print("The 'date' index is tz-aware.")

In [182]:
macro_data.info()

In [183]:
# 查看data中的索引date是否为tz-aware
try:
    data.index.tz_localize('UTC')
    print("The 'date' index is not tz-aware.")
except TypeError:
    print("The 'date' index is tz-aware.")
    print(macro_data.index.tz)

In [184]:
# 将macro_data根据日期与data合并
data = data.join(macro_data, how='left')
data.info()

In [185]:
# 分别从macro_data、data中取2020-01-02的bdi数据验证是否合并正确
print(data.loc['2020-01-02', 'bdi'])
print(macro_data.loc['2020-01-02', 'bdi'])

# 情绪数据

AK恐惧贪婪指数只能获取到22年后，暂不考虑了

In [186]:
def calculate_ar(data, period=26):
    """
    计算AR（人气指标）。

    参数:
    - data: 包含'high', 'low', 'open'列的DataFrame。
    - period: 计算指标的周期，默认为26天。

    返回:
    - ar: DataFrame，包含计算周期内的AR值。
    """
    ar_numerator = (data['high'] - data['open']).rolling(window=period).sum()
    ar_denominator = (data['open'] - data['low']).rolling(window=period).sum()
    ar = (ar_numerator / ar_denominator) * 100
    return ar


In [187]:
def calculate_br(data, period=26):
    """
    计算BR（意愿指标）。

    参数:
    - data: 包含'high', 'low', 'close'列的DataFrame。
    - period: 计算指标的周期，默认为26天。

    返回:
    - br: DataFrame，包含计算周期内的BR值。
    """
    br_numerator = (data['high'] - data['close'].shift()).rolling(window=period).sum()
    br_denominator = (data['close'].shift() - data['low']).rolling(window=period).sum()
    br = (br_numerator / br_denominator) * 100
    return br


In [188]:
# 加入到data中
data['AR'] = calculate_ar(data)
data['BR'] = calculate_br(data)

data.info()

# 特征工程

In [189]:
data

In [190]:
# 去除有空值的行
data.dropna(inplace=True)


In [191]:
data.describe()

## 数据“打包”，为LSTM做准备

In [192]:
from sklearn.preprocessing import StandardScaler

def pack_data_with_scaling(data, input_columns, output_column, window_size):
    """
    将DataFrame中的数据打包成LSTM的输入和输出，并对每个窗口的输入数据进行标准化处理。
    """
    X = []
    y = []
    scaler = StandardScaler()
    
    for i in range(len(data) - window_size):
        window_data = data[input_columns].iloc[i:i + window_size] #python的区间是左闭右开
        scaled_window_data = scaler.fit_transform(window_data)
        X.append(scaled_window_data)  #只对输入数据进行标准化处理
        y.append(data[output_column].iloc[i + window_size]) 
    
    return np.array(X), np.array(y)

# 选择用于预测的特征列
# input_columns = ['close', 'volume', 'log_return', 'MA_5', 'MA_10', 'MA_20', 'MA_60', 'MA_120',
#                  'EMA_5', 'EMA_10', 'EMA_20', 'EMA_60', 'EMA_120',
#                  'RSI', 'ATR', 'Bollinger_Width', 'VWAP', 'bdi', 'AR', 'BR']
input_columns = data.columns.tolist()
print(input_columns)
output_column = 'avg_daily_return_'+str(T)
# 打包数据
window_size=20
X, y = pack_data_with_scaling(data, input_columns, output_column, window_size)

# 说明：X的形状为(样本数量, 窗口大小, 特征数量)，y的形状为(样本数量,)。
# print这些信息，要对应到“样本数量：”
print(X.shape, y.shape)
print('样本数量：', X.shape[0])
print('窗口大小：', X.shape[1])
print('特征数量：', X.shape[2])

## 压缩X，与y合并

In [207]:
def flatten_and_combine_data(X, y, input_columns, output_column, window_size):
    """
    展开时间序列数据并将特征和目标变量合并为一个DataFrame。
    
    参数:
    - X: 时间序列数据的三维numpy数组，形状为(样本数, 时间步数, 特征数)。
    - y: 目标变量的一维numpy数组，形状为(样本数,)。
    - input_columns: 特征列名称的列表。
    - output_column: 输出列的名称。
    - window_size: 窗口大小，即每个样本包含的时间步数。
    
    返回:
    - 一个DataFrame，其中包含展开的特征和目标变量。
    -   一个特征名称的列表。
    """
    
    # 将X压平
    X_flattened = X.reshape(X.shape[0], -1)  # 将X压平

    # 生成展开后的特征名称
    feature_names = [f"{feature}_t-{t}" for feature in input_columns for t in range(window_size, 0, -1)]
    
    # 将X_flattened转换为DataFrame
    X_df = pd.DataFrame(X_flattened, columns=feature_names)
    
    # 将y转换为DataFrame
    y_df = pd.DataFrame(y, columns=[output_column])
    
    # 合并X_df和y_df
    data_combined = pd.concat([X_df, y_df], axis=1)
    # 加上列名
    data_combined.columns = feature_names + [output_column]
    return data_combined, feature_names

def mean_and_combine_data(X, y, input_columns, output_column, window_size):
    """
    计算每个窗口的特征的均值，并将特征和目标变量合并为一个DataFrame。
    
    参数:
    - X: 时间序列数据的三维numpy数组，形状为(样本数, 时间步数, 特征数)。
    - y: 目标变量的一维numpy数组，形状为(样本数,)。
    - input_columns: 特征列名称的列表。
    - output_column: 输出列的名称。
    - window_size: 窗口大小，即每个样本包含的时间步数。
    
    返回:
    - 一个DataFrame，其中包含每个窗口的特征的均值和目标变量。
    -  一个特征名称的列表。
    """
    
    # 计算每个窗口的特征的均值
    X_mean = X.mean(axis=1)
    
    # 将X_mean转换为DataFrame
    X_mean_df = pd.DataFrame(X_mean, columns=input_columns)
    
    # 将y转换为DataFrame
    y_df = pd.DataFrame(y, columns=[output_column])
    
    # 合并X_mean_df和y_df
    data_combined = pd.concat([X_mean_df, y_df], axis=1)
    
    # input_columns中的特征名称加上'_mean'
    input_columns = [f"{col}_mean" for col in input_columns]
    
    # 加上列名
    data_combined.columns = input_columns + [output_column]
    return data_combined, input_columns


data_combined, feature_names = mean_and_combine_data(X, y, input_columns, output_column, window_size)

data_combined.info()

In [198]:

# 将data_combined中的非数值列转换为数值列，将不能转换的列保存到一个列表

## 分析数据

### 相关性分析

In [208]:
output_column

In [209]:
import seaborn as sns
import matplotlib.pyplot as plt

# 计算相关系数矩阵
corr_matrix = data_combined.corr()

In [210]:
# 查看目标变量与其他特征的相关性
target_corr = corr_matrix[output_column].sort_values(ascending=False)
target_corr


In [215]:
# 计算相关系数，R2
R2 = target_corr**2
R2

# 查看相关系数较小的特征
low_corr_features = target_corr[target_corr.abs() < 0.1]
low_corr_features

In [216]:
# 设置matplotlib的参数，以确保图形能够适当显示
plt.figure(figsize=(20, 15))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Feature Correlation Matrix")
plt.show()

In [217]:
data_combined.corr()

In [2]:
from data.get_data import get_common_data

data = get_common_data( 'SHSE.510300', '2019-02-28', '2020-02-28',3)
data